# Defect Handling History - Apartment 7

This notebook generates the "Defect Handling History" chart, tracking the number of pending defects over time.
Updates:
- Counts defects per report snapshot (omitted = fixed).
- Identifies distinct defects by (Category, Location, Description).

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sqlite3
import pandas as pd
import os

%matplotlib inline

# Connect to DB
db_path = r'c:\Users\yoel\constructor\prisma\dev.db'
conn = sqlite3.connect(db_path)

STATUS_MAP = {
    'COMPLETED': 'OK',
    'COMPLETED_OK': 'OK',
    'DEFECT': 'DEFECT',
    'NOT_OK': 'DEFECT',
    'IN_PROGRESS': 'PENDING',
    'PENDING': 'PENDING',
}

def plot_defect_history(apt_num):
    print(f"Generating Defect Handling History for Apartment {apt_num}...")
    
    query = f"""
    SELECT 
        r.reportDate,
        a.number as apartment_number,
        wi.category,
        wi.status,
        wi.location,
        wi.description
    FROM WorkItem wi
    JOIN Report r ON wi.reportId = r.id
    JOIN Apartment a ON wi.apartmentId = a.id
    WHERE a.number = '{apt_num}'
    ORDER BY r.reportDate ASC
    """
    
    df = pd.read_sql_query(query, conn)
    if df.empty:
        print("No data found.")
        return

    df['reportDate'] = pd.to_datetime(df['reportDate'], unit='ms')
    df['state'] = df['status'].map(STATUS_MAP).fillna('INFO')
    
    # Identify defects in each report
    defects_only = df[df['state'] == 'DEFECT'].copy()
    
    # Count defects per Report and Category
    # Note: We assume that within a single report, each row is a distinct defect.
    history_counts = defects_only.groupby(['reportDate', 'category']).size().reset_index(name='pending_defects')
    
    # Ensure all report dates are represented for all categories
    report_dates = sorted(df['reportDate'].unique())
    categories = sorted(df['category'].unique())
    
    full_index = pd.MultiIndex.from_product([report_dates, categories], names=['reportDate', 'category'])
    df_history = history_counts.set_index(['reportDate', 'category']).reindex(full_index, fill_value=0).reset_index()
    
    plt.figure(figsize=(14, 8))
    has_data = False
    for category in categories:
        cat_data = df_history[df_history['category'] == category]
        if cat_data['pending_defects'].sum() > 0:
             plt.plot(cat_data['reportDate'], cat_data['pending_defects'], marker='o', label=category, linewidth=2)
             has_data = True
    
    if not has_data:
        print("No defects to plot.")
        return
        
    title_text = f"היסטוריית טיפול בליקויים - דירה {apt_num}"
    plt.title(title_text, fontsize=16, fontweight='bold')
    plt.xlabel('Date')
    plt.ylabel('Pending Defects')
    plt.grid(True, linestyle='--', alpha=0.3)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

plot_defect_history('7')